In [46]:
# <-- Import libraries, custom functions, and load configuration & datasets <-- #

import yaml
import datetime as dt
import re
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import seaborn as sns

from statsmodels.multivariate.manova import MANOVA
from scipy import stats
from scipy.stats import pearsonr, boxcox, chi2_contingency, shapiro, probplot
from scipy.stats.contingency import association

sns.set(style="whitegrid")

# <-- Imports custom preprocessing functions from 'functions.py' <-- #

# from functions import ()

# <-- Loads YAML configuration to dynamically reference CSV output files. <-- #

config = None  # <-- Initialize config
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

df_demo_control = pd.read_csv(config['output_data']['file4'])
df_web_data = pd.read_csv(config['output_data']['file5'])

In [47]:
df_web_data['date_time'] = pd.to_datetime(df_web_data['date_time'])

In [48]:
display(df_web_data.sort_values(by=['date_time'],ascending=True))
print(df_web_data.dtypes)

,client_id,visitor_id,visit_id,process_step,date_time
0,9088444,242404224_96732670250,423038079_46067236368_400417,step_3,2017-03-15 00:03:03
1,7179755,167765295_97487764427,264484508_5982901710_928530,start,2017-03-15 00:19:28
2,7179755,167765295_97487764427,264484508_5982901710_928530,step_1,2017-03-15 00:20:50
3,7179755,167765295_97487764427,264484508_5982901710_928530,step_2,2017-03-15 00:22:52
4,7179755,167765295_97487764427,264484508_5982901710_928530,step_3,2017-03-15 00:23:47
...,...,...,...,...,...
341041,1104891,710002770_61911521918,597284396_96145314289_460044,step_1,2017-04-30 23:57:32
341042,5402972,481155336_6881867780,989795424_84700610095_958376,step_3,2017-04-30 23:57:56
341043,8007865,502318665_28702751895,104145193_36067138148_118971,start,2017-04-30 23:57:59
341044,8007865,502318665_28702751895,104145193_36067138148_118971,start,2017-04-30 23:58:00


client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object


In [49]:
display(df_demo_control)
print(df_demo_control.dtypes)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,clnt_age_quantile,tenure_quantile
0,2304905,7.0,94.0,58.0,2.0,110860.30,6.0,9.0,Control,A3,T2
1,5126305,12.0,145.0,33.0,2.0,103671.75,0.0,3.0,Control,A2,T3
2,3727881,5.0,71.0,30.5,2.0,23915.60,0.0,3.0,Control,A1,T1
3,272934,5.0,66.0,58.5,2.0,27021.42,2.0,5.0,Control,A3,T1
4,285619,30.0,369.0,67.5,2.0,299388.72,3.0,6.0,Control,A4,T4
...,...,...,...,...,...,...,...,...,...,...,...
23521,6662941,7.0,84.0,36.0,2.0,83101.84,4.0,7.0,Control,A2,T2
23522,2787512,23.0,283.0,48.0,3.0,123618.60,1.0,4.0,Control,A3,T4
23523,9932367,23.0,280.0,57.0,2.0,94466.55,6.0,9.0,Control,A3,T4
23524,4079649,23.0,276.0,62.5,3.0,57575.28,4.0,7.0,Control,A4,T4


client_id              int64
clnt_tenure_yr       float64
clnt_tenure_mnth     float64
clnt_age             float64
num_accts            float64
bal                  float64
calls_6_mnth         float64
logons_6_mnth        float64
Variation             object
clnt_age_quantile     object
tenure_quantile       object
dtype: object


In [50]:
df_web_data[df_web_data['client_id'] == 4079649].sort_values(by='date_time')

,client_id,visitor_id,visit_id,process_step,date_time
201824,4079649,135423080_71753269572,155993046_1538887816_806756,start,2017-04-10 12:16:03
201834,4079649,135423080_71753269572,155993046_1538887816_806756,step_1,2017-04-10 12:16:45
201846,4079649,135423080_71753269572,155993046_1538887816_806756,step_2,2017-04-10 12:17:39
201856,4079649,135423080_71753269572,155993046_1538887816_806756,step_3,2017-04-10 12:18:03
201876,4079649,135423080_71753269572,155993046_1538887816_806756,confirm,2017-04-10 12:19:52


In [51]:
# Keep only client_id to avoid duplicate column proliferation
df_web_data_control = (df_web_data.merge(df_demo_control[['client_id']].drop_duplicates(), on='client_id', how='inner'))

print(f"Original web rows: {len(df_web_data):,}")
print(f"Filtered web rows (control only): {len(df_web_data_control):,}")

Original web rows: 341,046
Filtered web rows (control only): 100,569


- df_web_data_control['client_id']: Extracts the client_id column (a pandas Series) from the dataframe df_web_data_control.
- set(df_web_data_control['client_id']): Converts that Series to a Python set, removing duplicates. So you now have the unique client IDs that appear in the “web data” control dataframe.
- Same for set(df_demo_control['client_id']): the unique client IDs in the demo_control dataframe.

**.issubset(...)**
- Returns True if every element in the left-hand set (web data clients) appears in the right-hand set (demo clients). In other words:
- All client_ids present in df_web_data_control must also exist in df_demo_control.

**assert <condition>**
- If the condition is True, nothing happens (code continues).
- If it is False, Python raises an AssertionError, stopping execution (unless caught).

In [52]:
# 1. Ensure no client outside control group
assert set(df_web_data_control['client_id']).issubset(set(df_demo_control['client_id']))

# 2. Count distinct control clients represented in web data
print("Distinct control clients with activity:", df_web_data_control['client_id'].nunique())

# 3. How many control clients have zero web records?
control_without_web = (set(df_demo_control['client_id']) - set(df_web_data_control['client_id']))
print("Control clients without any web activity:", len(control_without_web))

Distinct control clients with activity: 18011
Control clients without any web activity: 5515


In [53]:
df_web_data_control.to_csv(config['output_data']['file6'], index=False, sep=",", encoding="utf-8")

In [54]:
df_web_data_control['client_id'].unique()

array([7179755, 9584408, 4856478, ..., 2091122,  292768, 4445887],
      shape=(18011,))

In [55]:
# Example of a completed & successful process (No error) 
df_web_data_control[df_web_data_control['client_id'] == 4079649].sort_values(by='date_time')

,client_id,visitor_id,visit_id,process_step,date_time
61034,4079649,135423080_71753269572,155993046_1538887816_806756,start,2017-04-10 12:16:03
61036,4079649,135423080_71753269572,155993046_1538887816_806756,step_1,2017-04-10 12:16:45
61041,4079649,135423080_71753269572,155993046_1538887816_806756,step_2,2017-04-10 12:17:39
61044,4079649,135423080_71753269572,155993046_1538887816_806756,step_3,2017-04-10 12:18:03
61050,4079649,135423080_71753269572,155993046_1538887816_806756,confirm,2017-04-10 12:19:52


In [56]:
# Example of complted operation with step-backs (it has reached process_step = 'confirm')
df_web_data_control[df_web_data_control['client_id'] == 4033851].sort_values(by='date_time')

,client_id,visitor_id,visit_id,process_step,date_time
37052,4033851,762728880_76361333336,949661017_22392791362_127391,start,2017-04-05 12:04:00
37078,4033851,762728880_76361333336,949661017_22392791362_127391,step_1,2017-04-05 12:04:56
37129,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:06:37
37188,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:08:56
37228,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:10:25
37229,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:25
37230,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:26
37231,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:10:26
37534,4033851,762728880_76361333336,949661017_22392791362_127391,start,2017-04-05 12:24:04
37535,4033851,762728880_76361333336,949661017_22392791362_127391,step_1,2017-04-05 12:24:12


In [57]:
# Example of a client with multiple process: 
# --> First process is a failed process that failed at the process_step = start
# --> Second process completd sucessfully with out step-backs

df_web_data_control[df_web_data_control['client_id'] == 5305116].sort_values(by='date_time')

,client_id,visitor_id,visit_id,process_step,date_time
34099,5305116,48895020_52187827152,454423043_73789478140_859004,start,2017-04-05 09:02:49
34100,5305116,48895020_52187827152,454423043_73789478140_859004,start,2017-04-05 09:03:22
90949,5305116,87196875_82592805389,69164930_20082199445_40666,start,2017-04-20 09:25:23
90950,5305116,87196875_82592805389,69164930_20082199445_40666,step_1,2017-04-20 09:25:58
90951,5305116,87196875_82592805389,69164930_20082199445_40666,step_2,2017-04-20 09:26:41
90954,5305116,87196875_82592805389,69164930_20082199445_40666,step_3,2017-04-20 09:27:44
90965,5305116,87196875_82592805389,69164930_20082199445_40666,confirm,2017-04-20 09:30:11


In [58]:
# Example of a failed process on start
df_web_data_control[df_web_data_control['client_id'] == 1197].sort_values(by='date_time')

,client_id,visitor_id,visit_id,process_step,date_time
25301,1197,753759429_54481946928,71862471_21202285428_848395,start,2017-04-01 17:16:24
25302,1197,753759429_54481946928,71862471_21202285428_848395,step_1,2017-04-01 17:16:28
25304,1197,753759429_54481946928,71862471_21202285428_848395,step_2,2017-04-01 17:16:34
25308,1197,753759429_54481946928,71862471_21202285428_848395,step_3,2017-04-01 17:17:25
25309,1197,753759429_54481946928,71862471_21202285428_848395,step_2,2017-04-01 17:17:36
25310,1197,753759429_54481946928,71862471_21202285428_848395,step_3,2017-04-01 17:17:42
25312,1197,753759429_54481946928,71862471_21202285428_848395,confirm,2017-04-01 17:17:59


In [59]:
# Example of a failed process on start
df_web_data_control[df_web_data_control['client_id'] == 1028].sort_values(by='date_time')

,client_id,visitor_id,visit_id,process_step,date_time
56041,1028,42237450_62128060588,557292053_87239438319_391157,start,2017-04-08 18:51:28
56045,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:52:17
56049,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:53:20
56050,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 18:53:29
56066,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04
56075,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:59:55
56077,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 19:00:15
56078,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 19:00:17
56079,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 19:00:26
